# Agentic AI MCP - Client Only

This notebook connects to a remote MCP server and runs agentic workflows.
The MCP server should be running on another machine (see `server.ipynb`).


## Step 1: Setup

In [1]:
# install if needed
# !uv pip install agentic-ai-mcp==0.5.3

import agentic_ai_mcp

print(f"Version: {agentic_ai_mcp.__version__}")

Version: 0.5.3


## Step 2: Make Sure To Have A Valid API_KEY

The LLM runs on this machine, so you need the API key here.
- For **Anthropic** (default): Set `ANTHROPIC_API_KEY`
- For **OpenAI**: Set `OPENAI_API_KEY`

In [ ]:
import os

from dotenv import load_dotenv

load_dotenv()

# check for API keys
if os.getenv("ANTHROPIC_API_KEY"):
    print("ANTHROPIC_API_KEY is set")
else:
    print("WARNING: ANTHROPIC_API_KEY not found")

if os.getenv("OPENAI_API_KEY"):
    print("OPENAI_API_KEY is set")
else:
    print("INFO: OPENAI_API_KEY not found (optional, only needed for OpenAI provider)")

## Step 3: Connect to Remote MCP Server

Use the `mcp_url` parameter to connect to an existing MCP server.

In [4]:
from agentic_ai_mcp import AgenticAI

# connect to remote MCP server (change this to your server's URL)
MCP_SERVER_URL = "http://127.0.0.1:8888/mcp"

ai = AgenticAI(mcp_url=MCP_SERVER_URL, verbose=True)  # client-only mode: connect to existing server
ai_openai = AgenticAI(
    mcp_url=MCP_SERVER_URL,
    provider="openai",
    model="gpt-4o-mini",  # or "gpt-4o", "gpt-4-turbo", etc.
    verbose=True
)

print(f"Client configured to connect to: {MCP_SERVER_URL}")

Client configured to connect to: http://127.0.0.1:8888/mcp


## Step 4a: Run Agent (Simple Task)

In [5]:
result = await ai.run("Calculate 2+1, then use the result as the number of times to greet 'Alice'.")
print()
print(">>>")
print()
print(result)

Loading tools from: http://127.0.0.1:8888/mcp
Loaded tools: ['add', 'multiply', 'greet']

PROMPT: Calculate 2+1, then use the result as the number of times to greet 'Alice'.



/mnt/jawad_cnms_s1/jawad_server/code/20260221_code_agentic_ai/agentic-ai-mcp/src/agentic_ai_mcp/agentic.py:185: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  self._agent = create_react_agent(


STEP 1: add({'a': 2, 'b': 1})
  → CallToolResult(content=[TextContent(type='text', text='{"result":3}', annotations=None, meta=None)], structured_content={'result': 3}, meta=None, data={'result': 3}, is_error=False)

STEP 2: greet({'name': 'Alice', 'times': 3})
  → CallToolResult(content=[TextContent(type='text', text='{"result":"Hello, Alice! Hello, Alice! Hello, Alice! "}', annotations=None, meta=None)], structured_content={'result': 'Hello, Alice! Hello, Alice! Hello, Alice! '}, meta=None, data={'result': 'Hello, Alice! Hello, Alice! Hello, Alice! '}, is_error=False)

RESULT: Perfect! The calculation 2+1 equals 3, and Alice has been greeted 3 times: **"Hello, Alice! Hello, Alice! Hello, Alice!"**


>>>

Perfect! The calculation 2+1 equals 3, and Alice has been greeted 3 times: **"Hello, Alice! Hello, Alice! Hello, Alice!"**


## Step 4b: Run Agent (Complex Task)

In [6]:
result = await ai.run_with_planning(
    "Calculate (((0+2) x (1+1)) + ((0+1) x (1+1))), then use the result as the number of times to greet 'Bob'."
)
print()
print(">>>")
print()
print(result)


PLANNING MODE
TASK: Calculate (((0+2) x (1+1)) + ((0+1) x (1+1))), then use the result as the number of times to greet 'Bob'.

PLAN:
  1. Use add(a=0, b=2) to calculate the first inner sum
  2. Use add(a=1, b=1) to calculate the second inner sum
  3. Use multiply(a=result from step 1, b=result from step 2) to multiply the first two results
  4. Use add(a=0, b=1) to calculate the third inner sum
  5. Use multiply(a=result from step 4, b=result from step 2) to multiply the third and second results
  6. Use add(a=result from step 3, b=result from step 5) to add the two products together
  7. Use greet(name="Bob", times=result from step 6) to greet Bob

EXECUTING STEP 1/7: Use add(a=0, b=2) to calculate the first inner sum
----------------------------------------


/mnt/jawad_cnms_s1/jawad_server/code/20260221_code_agentic_ai/agentic-ai-mcp/src/agentic_ai_mcp/workflows/planning.py:149: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  step_agent = create_react_agent(llm, tools)


  TOOL CALL 1: add({'a': 0, 'b': 2})
    → CallToolResult(content=[TextContent(type='text', text='{"result":2}', annotations=None, meta=None)], structured_content={'result': 2}, meta=None, data={'result': 2}, is_error=False)
  AI RESPONSE: The first inner sum is **2**. The result of add(a=0, b=2) is 2.
----------------------------------------
STEP 1 COMPLETE

EXECUTING STEP 2/7: Use add(a=1, b=1) to calculate the second inner sum
----------------------------------------
  TOOL CALL 1: add({'a': 1, 'b': 1})
    → CallToolResult(content=[TextContent(type='text', text='{"result":2}', annotations=None, meta=None)], structured_content={'result': 2}, meta=None, data={'result': 2}, is_error=False)
  AI RESPONSE: Perfect! **Step 2: The second inner sum is 2.** The result of add(a=1, b=1) is **2**.

So far we have:
- First inner sum: 2
- Second inner sum: 2
----------------------------------------
STEP 2 COMPLETE

EXECUTING STEP 3/7: Use multiply(a=result from step 1, b=result from step 2) to m